In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy import signal
from scipy.stats import spearmanr
import warnings
# 禁用所有警告
warnings.filterwarnings("ignore")
import os
import sys
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
from arch import arch_model
import matplotlib as mpl
from statsmodels.stats.diagnostic import acorr_ljungbox #白噪声检验
from statsmodels.graphics.tsaplots import plot_predict
from sklearn.metrics import mean_squared_error
#######读取数据等
from statsmodels.tsa.stattools import adfuller
# 设置使用中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 请替换为你系统中的中文字体
plt.rcParams['axes.unicode_minus'] = False  # 显示负号
from scipy.stats import norm

In [ ]:
anthophyllum=pd.read_excel("C:/Users/韩甄泽/OneDrive/桌面/2023年秋数据科学导论大作业/附件3_项目三数据集/merged_data_pl1_huaye.xlsx")
cauliflower=pd.read_excel("C:/Users/韩甄泽/OneDrive/桌面/2023年秋数据科学导论大作业/附件3_项目三数据集/merged_data_pl2_huacai.xlsx")
tubers=pd.read_excel("C:/Users/韩甄泽/OneDrive/桌面/2023年秋数据科学导论大作业/附件3_项目三数据集/merged_data_pl3_ssgj.xlsx")
eggplant=pd.read_excel("C:/Users/韩甄泽/OneDrive/桌面/2023年秋数据科学导论大作业/附件3_项目三数据集/merged_data_pl4_qie.xlsx")
capsicum=pd.read_excel("C:/Users/韩甄泽/OneDrive/桌面/2023年秋数据科学导论大作业/附件3_项目三数据集/merged_data_pl5_lajiao.xlsx")
mushroom=pd.read_excel("C:/Users/韩甄泽/OneDrive/桌面/2023年秋数据科学导论大作业/附件3_项目三数据集/merged_data_pl6_shiyongjun.xlsx")

In [ ]:
#分别画出ACF(自相关)和PACF（偏自相关）图像
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
 
def diff(df,col):
    font = {"size":15,
       "family":"fangsong"}
    plt.rc("font",**font)
    plt.rcParams['axes.unicode_minus']=False
    
    df["diff_1"] = df[col].diff(1)  #一阶差分
    df["diff_2"] = df["diff_1"].diff(1)  #二阶差分
    
    #平稳数据折线图
    plt.figure(figsize=(12,8))
    plt.subplot(3,1,1)
    plt.plot(df[col].values,label="源数据")
    plt.xlim(0,120)
    plt.legend()
    plt.subplot(3,1,2)
    plt.plot(df["diff_1"].values,c="darkgreen",label="一阶差分")
    plt.plot([0,120],[0,0],"--",c = "grey")
    plt.xlim(0,120)
    plt.legend()
    plt.subplot(3,1,3)
    plt.plot(df["diff_2"].values,c="tomato",label="二阶差分")
    plt.plot([0,120],[0,0],"--",c = "grey")
    plt.xlim(0,120)
    plt.legend()
    plt.show()
    
    #ACF PACF
    print("-"*50,"未平稳数据ACF与PACF","-"*50)
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(211)
    fig = plot_acf(df[col], lags=40,ax = ax1)
    ax2 = fig.add_subplot(212)
    plot_pacf(df[col], lags=40,ax = ax2)
    plt.show()
    
    #一阶差分后的ACF PACF
    print("-"*50,"一阶差分数据ACF与PACF","-"*50)
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(211)
    fig = plot_acf(df["diff_1"][1:].values, lags=40,ax = ax1)
    ax2 = fig.add_subplot(212)
    plot_pacf(df["diff_1"][1:], lags=40,ax = ax2)
    plt.show()
    #二阶差分后的ACF PACF
    print("-"*50,"二阶差分数据ACF与PACF","-"*50)
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(211)
    fig = plot_acf(df["diff_2"][1:].values, lags=40,ax = ax1)
    ax2 = fig.add_subplot(212)
    plot_pacf(df["diff_2"][1:], lags=40,ax = ax2)
    plt.show()

In [ ]:
#非白噪声检验函数
def tsplot(y, lags=None, figsize=(10, 8), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):
        fig = plt.figure(figsize=figsize)
        # mpl.rcParams['font.family'] = 'Ubuntu Mono'
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        qq_ax = plt.subplot2grid(layout, (2, 0))
        pp_ax = plt.subplot2grid(layout, (2, 1))

        y.plot(ax=ts_ax)
        ts_ax.set_title('Time Series Analysis Plots')
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5)
        sm.qqplot(y, line='s', ax=qq_ax)
        qq_ax.set_title('QQ Plot')
        stats.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)

        plt.tight_layout()
        plt.show()
    return

In [ ]:
anthophyllum['销售日期'] = pd.to_datetime(anthophyllum['销售日期'])
cauliflower['销售日期'] = pd.to_datetime(cauliflower['销售日期'])
tubers['销售日期'] = pd.to_datetime(tubers['销售日期'])
eggplant['销售日期'] = pd.to_datetime(eggplant['销售日期'])
capsicum['销售日期'] = pd.to_datetime(capsicum['销售日期'])
mushroom['销售日期'] = pd.to_datetime(mushroom['销售日期'])
anthophyllum = anthophyllum.groupby('销售日期')['销量(千克)'].sum().reset_index()
cauliflower = cauliflower.groupby('销售日期')['销量(千克)'].sum().reset_index()
tubers = tubers.groupby('销售日期')['销量(千克)'].sum().reset_index()
eggplant = eggplant.groupby('销售日期')['销量(千克)'].sum().reset_index()
capsicum = capsicum.groupby('销售日期')['销量(千克)'].sum().reset_index()
mushroom = mushroom.groupby('销售日期')['销量(千克)'].sum().reset_index()
df_list=[anthophyllum,cauliflower,tubers,eggplant,capsicum,mushroom]
print(anthophyllum)
print(cauliflower)
print(tubers)
print(eggplant)
print(capsicum)
print(mushroom)

In [ ]:
for df in df_list:
    df["diff_1"] = df['销量(千克)'].diff(1)  # 一阶差分
    df["diff_2"] = df['销量(千克)'].diff(2)  # 二阶差分
    print('*'*100)
    print('\n')
    diff(df,"销量(千克)")

In [ ]:

for df in df_list:
    ###########ADF检验
    adftest = adfuller(df['销量(千克)'], autolag='AIC')
    print (adftest)
    # 进行差分
    ads = pd.Series(df['销量(千克)'].values, index=df['销售日期'])
    print(ads.head())
    ads_diff = ads.diff(1)
    ads_diff = ads_diff.dropna()
    print(ads_diff.head())
    adftest = adfuller(ads_diff, autolag='AIC')
    print (adftest)
    ###########纯随机性检验（白噪声检验）
    p_value = acorr_ljungbox(ads_diff, lags=1)
    print(p_value)
    #信息准则确定p,d,q 这里选取AIC
    best_aic = np.inf
    best_order = (0,0,0)
    best_mdl = smt.ARIMA(ads_diff, order=(0,0,0)).fit()
#一般阶数不超过整体数据的十分之一，因此这里采用穷举法。分别从0~10取p、q。
    pq_rng = range(10) # [0,1,2,3,4]
    d_rng = range(3) # [0,1] ARMA
    for i in pq_rng:
         for j in pq_rng:
             try:

                tmp_mdl = smt.ARIMA(df['销量(千克)'], order=(i,1,j)).fit()
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic:
                    best_aic = tmp_aic
                    best_order = (i, 1, j)
                    best_mdl = tmp_mdl
             except: continue


    print('aic: {:6.5f} | order: {}'.format(best_aic, best_order))
    tsplot(best_mdl.resid, lags=8)
    best_mdl.summary()
    resid=best_mdl.resid  #残差
    plt.figure(figsize=(12,8))
    plt.plot(resid)
    plt.show()
    stats.normaltest(resid)#检验序列残差是否为正态分布    <  0.05  拒绝原假设 认为残差符合正态分布
    stats.probplot(resid, dist="norm", plot=plt)
    plt.show()
    plt.hist(resid,bins=50)
    plt.show()
### 德宾-沃森（Durbin-Watson）检验
    print(sm.stats.durbin_watson(best_mdl.resid.values))
    n_steps = 7
# 生成预测
    forecast = best_mdl.get_forecast(steps=n_steps)


# Plotting
    plt.figure(figsize=(12, 6))

# Plotting original data
    plt.plot(df['销量(千克)'].values, label='原数据')
    plt.show()
# Plotting forecasted data
    plt.plot(np.arange(len(df['销量(千克)']), len(df['销量(千克)']) + n_steps),
         forecast.predicted_mean, color='red', label='预测数据')
    plt.legend()
    plt.show()
    forecast_values = best_mdl.predict(start=len(df['销量(千克)']), end=len(df['销量(千克)']) + n_steps-1, typ='levels')

    # 打印预测结果
    print(forecast_values)
    forecast_values = pd.to_excel("C:/Users/韩甄泽/OneDrive/桌面/2023年秋数据科学导论大作业/附件3_项目三数据集/merged_data.xlsx")
